# Machine Learning with Flowers

In [29]:
#Importing Code Hidden
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adamax

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [30]:
#Code Hidden
data = []
for dir, x, files in os.walk("flower_photos"):
    label = dir.split("/")[-1]
    for file in files:
        path = os.path.join(dir, file)
        data.append([path, label])

df = pd.DataFrame(data, columns=["file_path", "label"])
print(df)

                                             file_path   label
0       flower_photos/roses/16209331331_343c899d38.jpg   roses
1        flower_photos/roses/5777669976_a205f61e5b.jpg   roses
2      flower_photos/roses/4860145119_b1c3cbaa4e_n.jpg   roses
3       flower_photos/roses/15011625580_7974c44bce.jpg   roses
4     flower_photos/roses/17953368844_be3d18cf30_m.jpg   roses
...                                                ...     ...
3665     flower_photos/tulips/134143359_71fa8dd9a4.jpg  tulips
3666    flower_photos/tulips/3637371174_a8dfcc1b35.jpg  tulips
3667  flower_photos/tulips/6948239566_0ac0a124ee_n.jpg  tulips
3668    flower_photos/tulips/2834890466_1cf220fba1.jpg  tulips
3669   flower_photos/tulips/13953090784_0c7d7a904e.jpg  tulips

[3670 rows x 2 columns]


In [31]:
#Code Hidden
datagen = ImageDataGenerator(
    validation_split = 0.2
)

In [38]:
#Code Hidden
train_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = "file_path",
    y_col = "label",
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "training"
)

valid_gen = datagen.flow_from_dataframe(
    dataframe = df,
    x_col = "file_path",
    y_col = "label",
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True,
    subset = "validation"
)

print(train_gen.class_indices.items())

Found 2936 validated image filenames belonging to 5 classes.
Found 734 validated image filenames belonging to 5 classes.
dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])


In [33]:
#Code Hidden
base_model = EfficientNetB3(
    include_top= False, 
    weights='imagenet',
    input_shape=(224, 224, 3), 
    pooling='max'
)

model = tf.keras.models.Sequential([
    base_model,
    layers.BatchNormalization(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(rate=.45, seed=123),
    layers.Dense(5, activation='softmax')
])


model.compile(
    Adamax(learning_rate= .0001), 
    loss = 'categorical_crossentropy',
    metrics=[
        'acc', 
        tf.keras.metrics.Precision(), 
        tf.keras.metrics.Recall(), 
        tf.keras.metrics.AUC()
    ]
)

In [34]:
#Code Hidden
early_stopping = EarlyStopping(
    patience = 10,
    min_delta = 0,
    monitor = "val_loss",
    verbose = 0,
    restore_best_weights = True,
    baseline = None
)

In [27]:
#AI Training Code Hidden
model.fit(
    train_gen,
    validation_data = valid_gen,
    epochs = 5,
    validation_steps = None,
    shuffle = False,
    callbacks = early_stopping,
    verbose = 1
)

Epoch 1/5


/Users/Cohort23/Library/Python/3.10/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 323s 3s/step - acc: 0.4208 - auc_2: 0.7125 - loss: 1.7108 - precision_2: 0.4680 - recall_2: 0.3403 - val_acc: 0.1431 - val_auc_2: 0.4436 - val_loss: 2.7207 - val_precision_2: 0.1502 - val_recall_2: 0.1035
Epoch 2/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 316s 3s/step - acc: 0.7577 - auc_2: 0.9395 - loss: 0.6912 - precision_2: 0.8199 - recall_2: 0.6967 - val_acc: 0.1158 - val_auc_2: 0.3987 - val_loss: 3.7497 - val_precision_2: 0.1244 - val_recall_2: 0.1076
Epoch 3/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 331s 4s/step - acc: 0.8215 - auc_2: 0.9672 - loss: 0.5032 - precision_2: 0.8614 - recall_2: 0.7841 - val_acc: 0.1172 - val_auc_2: 0.3757 - val_loss: 4.4415 - val_precision_2: 0.1235 - val_recall_2: 0.1117
Epoch 4/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 316s 3s/step - acc: 0.8815 - auc_2: 0.9824 - loss: 0.3585 - precision_2: 0.9109 - recall_2: 0.8508 - val_acc: 0.1185 - val_auc_2: 0.3668 - val_loss: 4.9834 - val_precision_2: 0.1265 - val_recall_2: 0.1172
Epoch 5/5
92/92 ━━━━━━━━━━━━━━━━━━━━ 329s 

In [28]:
#Save Code Hidden
model.save("flowers.keras")

In [42]:
#Code Hidden
import cv2
from tensorflow.keras.saving import load_model
import numpy as np


model = load_model("flowers.keras")

input_image = cv2.imread("flower_photos/tulips/10686568196_b1915544a8.jpg")
input_image_resize = cv2.resize(input_image, (224,224))

input_image_scaled = input_image_resize/255
image_reshaped = np.reshape(input_image_scaled, [1, 224, 224, 3])

input_prediction = model.predict(image_reshaped)
print(train_gen.class_indices.items())
print("Index: ", np.argmax(input_prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
dict_items([('daisy', 0), ('dandelion', 1), ('roses', 2), ('sunflowers', 3), ('tulips', 4)])
Index:  0
